# First step
Go to conf.py and edit the path to where you want any outputs saved.

In [ ]:
# Import statements
from subprocess import Popen, PIPE
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import heasoftpy as hsp

from resolve import run_rslbratios
from classes import EventList, XRISMSource
from phot import make_image
from timing import make_lc
from spec import make_spectra_inc_rsp_and_nxb

# Re-run the pipeline
If the calibration database is updated it is reccomended to re-run the pipeline, so that's what I will do for resolve, as this calibration was updated in March

In [ ]:
instrument = 'RESOLVE'
indir = '/Users/jp735/Desktop/Geneva/data/3C273/000145000/'
outdir = '/Volumes/Seagate/xapipeline/'
steminputs = 'xa000145000'
stemoutputs = 'DEFAULT'
entrystage = '1' # Start from recalibrating unfiltered event files
exitstage = '2' # dont bother making quick look product
verify_input = 'no'

cmd = ("cd /Volumes/Seagate/; "
       "source ~/headas.sh; "
       "export HEADASNOQUERY= ; "
       "export HEADASPROMPT=/dev/null; "
       f"xapipeline indir={indir} outdir={outdir} steminputs={steminputs} stemoutputs={stemoutputs} "
       f"entry_stage={entrystage} exit_stage={exitstage} instrument={instrument} " 
       f"verify_input={verify_input} clobber=yes")

print(cmd)

In [ ]:
out, err = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE).communicate()
out = out.decode("UTF-8", errors='ignore')
err = err.decode("UTF-8", errors='ignore')
print(out)
print(err)

In [ ]:
# edit this to the new evt list wherever you reran the pipeline
evt_path = "/Volumes/Seagate/xapipeline/xa000145000rsl_p0px1000_cl.evt"

# Gain Reports
Before you start any product generation, you need to look at the resolve gain reports to double check if you need to exclude any pixels.

An example of some dodgy pixels in a gain report are below:

![image](imgs/dodgy_report.png)

You can find the gain reports here: https://heasarc.gsfc.nasa.gov/FTP/xrism/postlaunch/gainreports/

# Now you can make products

In [ ]:
OBSID = '000145000'
SRC_RA = 187.272
SRC_DEC = 2.054

# should come from your re-run of the pipeline
root_dir = '/Users/jp735/Desktop/Geneva/data/3c273'

evt = EventList(evt_path, OBSID, 'resolve', 'open', root_dir)
# should come from your re-run of the pipeline
evt.ehkfile = "/Volumes/Seagate/xapipeline/xa000145000.ehk"

In [ ]:
# This is applying the recommended screening in the quick start guide for XRISM
screened_evt = evt.screen(apply_rsl_rise_time=True, min_pi=600, status=[4])

# Make lightcurve, images, and spectra
Make sure to always exclude pixels 12 and 27, as well as any bad pixels identified in the gain report

In [ ]:
# lightcurve
# The pix_name argument is just a string for naming the files
make_lc(screened_evt, 
        exclude_pix=[12, 27, 29], 
        pix_name='no29'
        )

In [ ]:
make_image(screened_evt,        
           exclude_pix=[12, 27, 29], 
           pix_name='no29')

In [ ]:
# These are downloadable from the nxb section of the XRISM website
nxbfile = '/Users/jp735/Desktop/Geneva/data/nxb/merged_nxb_resolve_gtifix.evt'
nxbehkfile = '/Users/jp735/Desktop/Geneva/data/nxb/merged_reduced_rev3_fix2.ehk'

# This makes the arf and rmf which both take ages, so dont worry if 
# this step takes ~ 5-8 hours to run
# There is a make_spectra argument that you can use if you just
# want to quickly see a spectrum
make_spectra_inc_rsp_and_nxb(screened_evt, SRC_RA, SRC_DEC, 'L', 
                            nxbfile=nxbfile,
                            nxbehkfile=nxbehkfile,
                            numphoton=300000,
                            cortime=6,
                            apply_rsl_rise_time=True, min_energy=600, status=[4],
                            timefirst=730,
                            exclude_pix=[12, 27, 29], 
                            pix_name='no29'
                            )